<a href="https://colab.research.google.com/github/mileslilly2/Lecture-Notes-Prototype/blob/main/Extract_text_Pdf_with_clarfia_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installs and functions


In [2]:
%%capture
#!pip install --upgrade typing_extensions
!pip install clarifai
!pip install PyPDF2
!pip install pdfplumber
!pip install fitz
!pip install frontend
!pip install transformers torch Pillow requests
!pip install pdf2image
!pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain

# Set env var OPENAI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_dotenv()
!apt-get install -y poppler-utils


In [3]:
from google.colab import userdata

%env CLARIFAI_PAT = d2539baf2b9640f9993cd7411d5ff3e6

env: CLARIFAI_PAT=d2539baf2b9640f9993cd7411d5ff3e6


### Mount to Drive

In [4]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/books

/content/drive/MyDrive/books


In [6]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%late

## Text processing functions

#### PDF text extraction

In [7]:
# Python Template for PDF Summarization Tool - Steps 1 and 2

# Step 1: PDF Parsing
# You can use PyPDF2 or PDFMiner for this. Here's a basic example using PyPDF2.

import PyPDF2

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a given PDF file.
    :param pdf_path: Path to the PDF file.
    :return: Extracted text as a string.
    """
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Step 2: Text Processing
# Basic preprocessing of the text to remove unwanted characters, spaces, etc.

import re

def preprocess_text(text):
    """
    Preprocesses the extracted text.
    :param text: Extracted text from the PDF.
    :return: Preprocessed text.
    """
    # Remove special characters and multiple spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    #text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    return text.strip()




### compute token length

In [8]:
import tiktoken
from math import ceil

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens



def count_image_tokens(width: int, height: int):
    h = ceil(height / 512)
    w = ceil(width / 512)
    n = w * h
    total = 85 + 170 * n
    return total

print(num_tokens_from_string("Hello world, let's test tiktoken.", "gpt-3.5-turbo"))

9


## Image proccessing fuctions

### grab images

In [9]:
from pdf2image import convert_from_path
from PIL import Image
import os

def save_pdf_pages_as_images(pdf_path, output_folder):
    # Convert PDF to a list of images
    pages = convert_from_path(pdf_path, 300)

    # Create output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Save each page as an image
    for i, page in enumerate(pages):
        # Define image file name
        image_filename = os.path.join(output_folder, f"page_{i + 1}.jpg")

        # Save image
        page.save(image_filename, 'JPEG')

    print(f"Saved images to {output_folder}")


### Process and encode image Open AI

In [10]:
import os
import requests
import base64
from io import BytesIO

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def image_to_base64(pil_image):
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # or format="PNG" depending on your image
    return base64.b64encode(buffered.getvalue()).decode()




def analyze_image(prompt, image_path):
    # Retrieve your OpenAI API key from the environment variable
    openai_api_key = os.getenv("OPENAI_API_KEY")

    # Ensure the API key is available
    if not openai_api_key:
        raise ValueError("OpenAI API key is not set")

    # Getting the base64 string
    base64_image = encode_image(image_path)
    base64_image_2 = encode_image(image_path_2)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview", # Replace with the appropriate model name
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        return response.json()
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None



def analyze_image(prompt, image_path, image_path_2):
    # Retrieve your OpenAI API key from the environment variable
    openai_api_key = os.getenv("OPENAI_API_KEY")

    # Ensure the API key is available
    if not openai_api_key:
        raise ValueError("OpenAI API key is not set")

    # Getting the base64 string
    base64_image = encode_image(image_path)
    base64_image_2 = encode_image(image_path_2)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }

    payload = {
        "model": "gpt-4-vision-preview", # Replace with the appropriate model name
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image_2}"
                        }
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        return response.json()
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Use the function like this
# result = analyze_image("Describe this image", "path_to_image_1.jpg", "path_to_image_2.jpg")

# Usage example:
# api_key = "YOUR_OPENAI_API_KEY"
# image_path = "path_to_your_image.jpg"
# result = analyze_image(image_path, api_key)
# print(result)
print('pies 4 sure')

pies 4 sure


### Load images from dir

In [11]:
import os
from PIL import Image

def load_images_from_folder(folder_path):
    images_list = []
    # Ensure the files are sorted numerically
    image_files = sorted(os.listdir(folder_path), key=lambda x: int(x.split('_')[1].split('.')[0]))

    for filename in image_files:
        if filename.endswith('.jpg'):  # Check if the file is a JPEG image
            file_path = os.path.join(folder_path, filename)  # Full path to the image
            image = Image.open(file_path)  # Open the image
            images_list.append(image)  # Append the image to the list

    return images_list



### Caption the text using sf model

In [12]:
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

# Initialize the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

def generate_caption_from_image(image, text=None):
    """
    Generate a caption for an image.

    Parameters:
    - image: a PIL.Image.Image object
    - text: optional text prompt for conditional captioning

    Returns:
    - caption: the generated caption for the image
    """
    # Ensure the image is in RGB mode
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Perform image captioning
    if text:
        # Conditional image captioning
        inputs = processor(image, text, return_tensors="pt")
    else:
        # Unconditional image captioning
        inputs = processor(image, return_tensors="pt")

    # Generate the caption
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption




preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

## Summary functions

### Implement functions for Map-Reduce summeriztion techniques

In [14]:

from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate  # Assuming PromptTemplate needs to be imported
from typing import Iterator, AsyncIterator
from langchain.chains import LLMChain
#from openai import OpenAI

from langchain_openai import OpenAI


#from langchain.llms import LLMChain, ChatOpenAI  # Assuming LLMChain and ChatOpenAI need to be imported

# Initialize the language model
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

llm = OpenAI(openai_api_key=openai_api_key,model="gpt-3.5-turbo", temperature=0)


def chunk_doc(big_doc, chunk_size):
  # Example usage
  splitter = CharacterTextSplitter(
      chunk_size=chunk_size)  # Adjust max_characters as needed

  texts = splitter.create_documents([big_doc])

  return texts


def map_langchain(document_chunks):
  # Map prompt template
  map_template = """The following is a set of documents
  {docs}
  Based on this list of docs please identify the main themes
  Helpful Answer:"""
  map_prompt = PromptTemplate.from_template(map_template)
  map_chain = LLMChain(llm=llm, prompt=map_prompt)

  # MapReduceDocumentsChain setup
  map_reduce_chain = MapReduceDocumentsChain(
      llm_chain=map_chain,
      reduce_documents_chain=document_chunks,
      document_variable_name="docs",
      return_intermediate_steps=False)
  mapped_results = [map_chain.process(doc) for doc in document_chunks]
  return mapped_results


def reduce_langchain(split_docs):
  # Reduce prompt template
  reduce_template = """The following is set of summaries:
  {docs}
  Take these and distill it into a final consolidated summary of the main themes.
  Helpful Answer:"""
  reduce_prompt = PromptTemplate.from_template(reduce_template)
  reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

  # Reduce documents chain setup
  reduce_documents_chain = ReduceDocumentsChain(
      combine_documents_chain=
      split_docs,  # Assuming combine_documents_chain is defined elsewhere
      collapse_documents_chain=
      split_docs,  # Assuming collapse_documents_chain is defined elsewhere
      token_max=4000)
  return reduce_documents_chain


### image processing OpenAI code

In [15]:
from openai import OpenAI

def get_completion_with_images(text_summary, image1, image2):
    client = OpenAI()

    combined_prompt = (
        f"Here is a summary of the text related to two images: '{text_summary}'. "
        "Based on this summary, analyze the following images and explain how they relate to the text: "
    )

    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": combined_prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image1,
                        },
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image2,
                        },
                    },
                ],
            }
        ],
        max_tokens=300,
    )
    print(response.choices[0])


### Clarifai

In [56]:
from clarifai.client.model import Model

# Define a function to analyze images and print the model predictions
def analyze_images_clarifai(prompt, image_url):
    prompt = prompt

    # Retrieve OpenAI API key
    openai_api_key = userdata.get('OPENAI_API_KEY')

    # Initialize the model
    model = Model("https://clarifai.com/openai/chat-completion/models/gpt-4-vision")

       # Iterate over the image URLs and make predictions

    inference_params = dict(temperature= 0.2, max_tokens= 100, image_url= image_url,api_key= openai_api_key)



        # Model Predict
    model_prediction = model.predict_by_bytes(prompt.encode(), input_type='text', inference_params=inference_params)
    print(model_prediction.outputs[0].data.text.raw)

# Define the image URLs
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    "https://upload.wikimedia.org/wikipedia/commons/2/22/Turkish_Van_Cat.jpg"
]
image_url = 'https://upload.wikimedia.org/wikipedia/commons/2/22/Turkish_Van_Cat.jpg'

# Call the function with the list of image URLs
analyze_images_clarifai('What is in this image?', image_url)


This image shows a small white kitten with fluffy fur. The kitten has striking heterochromia, with one blue eye and one green eye, which is a condition where the irises of the eyes are different colors. The kitten appears to be sitting on or behind a woven object, possibly a basket, and is looking slightly upwards. The background is neutral and nondescript, putting the focus on the kitten's face and eyes.


In [42]:
from clarifai.client.model import Model

prompt = "What’s in this image?"

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

openai_api_key = userdata.get('OPENAI_API_KEY')

inference_params = dict(temperature=0.2, max_tokens=100, image_url=image_url, api_key = openai_api_key)

# Model Predict
model_prediction = Model("https://clarifai.com/openai/chat-completion/models/gpt-4-vision").predict_by_bytes(prompt.encode(), input_type="text", inference_params=inference_params)
print(model_prediction.outputs[0].data.text.raw)


The image shows a wooden boardwalk stretching out through a lush green wetland or grassland area. The boardwalk allows for a walking path above the grass, likely to protect the natural environment and to provide visitors with a way to explore without impacting the sensitive ecosystem. The sky is partly cloudy with blue skies and some white clouds, suggesting a pleasant day. The vegetation on either side of the boardwalk is dense and tall, indicating a healthy, thriving natural habitat. The scene is serene and picturesque,


## Run the code

First we will specify the directory or the url of the pdf we want to summarize. We also define the name of the output directory

In [ ]:
pdf = "/content/drive/MyDrive/books/Distributed Representations of Words and Phrases.pdf"
output_directory = 'dist_word_rep'

### First lets extract the images

### Extract the text from the pdf

In [ ]:
raw_text_extracted_from_pdf = extract_text_from_pdf(pdf)

In [ ]:
raw_text_extracted_from_pdf[0:100]

In [ ]:
cleaned_text = preprocess_text(raw_text_extracted_from_pdf)

In [ ]:
save_pdf_pages_as_images
# Example usage
save_pdf_pages_as_images(pdf, output_directory)


In [ ]:
imgs = load_images_from_folder(output_directory)

### lets caption all the image


In [ ]:
# Assuming 'loaded_images' is the list of PIL images
images_with_captions = []
for img in imgs:
    caption = generate_caption_from_image(img)
    images_with_captions.append((img, caption))
    print(caption)

In [ ]:
print(images_with_captions[0][0])

### subset the image caption tuples for processing

In [ ]:
images_for_vision_processing = []
for image in images_with_captions:
  if 'diagram' in image[1] or 'graph' in image[1]:
    images_for_vision_processing.append(image[0])


In [ ]:
print(images_for_vision_processing)

### Encode the images for the OpenAI api

In [ ]:
encoded_images = []
encoded_image = 0
for image in images_for_vision_processing:
  encoded_image = image_to_base64(image)
  encoded_images.append(encoded_image)

### Map-Reduce summary method

In [ ]:
doc_chunks = chunk_doc(cleaned_text, 2000)
mapped_chunks = map_langchain(doc_chunks)
final_text_summary = reduce_langchain(mapped_chunks)

In [ ]:
doc_chunks[0]

In [ ]:
print(final_text_summary)

### this is a placeholder. I think I can add this to a document object with images and text. Also the adobie api thing

In [ ]:
prompt = """The following is set of summaries:
  {docs}
  Take these and distill it into a final consolidated summary of the main themes.
  Helpful Answer:"""

In [ ]:
analyze_images_clarifai(prompt, encoded_images)

In [ ]:

num_loops = len(encoded_images)


In [ ]:
image_summary = ''
for i in range(num_loops -1 , 2):
  text = get_completion_with_images(final_text_summary, encoded_images[i], encoded_images[i+1])
  image_summary

In [ ]:
full_summary = image_summary + full_text_summary

In [ ]:
file_path = '/content/drive/MyDrive/books/dist_word.txt'  # Replace with your desired path and file name

with open(file_path, 'w') as file:
    file.write(full_summary)  # Assuming 'text' contains the data you want to write
